In [1]:
# Set display to the full length

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
!pip install pysradb

In [27]:
import pandas as pd
accession_list_path = 'current.txt'
metadata_file_extension = 'md'

In [4]:
# Generate a set of files containing metadata for SRA datasets
# This relies on current.txt as the source of accessions

with open(accession_list_path,'r') as f:
    for line in f:
        srr = line.rstrip()
        cmd = "pysradb metadata {0} --detailed --expand --saveto {0}.{1}".format(srr,metadata_file_extension)
        !{cmd}

/home/anton/anaconda3/envs/var_nb/lib/python3.8/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/home/anton/anaconda3/envs/var_nb/lib/python3.8/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/home/anton/anaconda3/envs/var_nb/lib/python3.8/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/home/anton/anaconda3/envs/var_nb/lib/python3.8/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/home/anton/anaconda3/en

In [28]:
# SRR and ERR accessions contain different columns
# Will populate two different dataframes as a result
# They are later reconciled into one

import os
srr_df = pd.DataFrame()
err_df = pd.DataFrame()
with open(accession_list_path,'r') as f:
    for line in f:
        srr = line.rstrip()
        filename = '{0}.{1}'.format(srr,metadata_file_extension)
        if os.path.isfile(filename):
            tdf = pd.read_csv(filename,sep='\t')
            if srr.startswith('SRR'):
                srr_df = pd.concat([srr_df,tdf], ignore_index=True)
            elif srr.startswith('ERR'):
                err_df = pd.concat([err_df,tdf], ignore_index=True)
        else:
            print('{} does not exist'.format(filename))

ERR4082721.md does not exist
ERR4082720.md does not exist
ERR4082718.md does not exist
ERR4082717.md does not exist
ERR4082716.md does not exist
ERR4082715.md does not exist
ERR4082713.md does not exist
ERR4082179.md does not exist
ERR4082110.md does not exist
SRR11314339.md does not exist
SRR11300931.md does not exist
SRR11059940.md does not exist
SRR11059941.md does not exist
SRR11059942.md does not exist
SRR11059943.md does not exist
SRR11059944.md does not exist
SRR11059945.md does not exist
SRR11059946.md does not exist
SRR11059947.md does not exist


In [29]:
err_df.columns

Index(['study_accession', 'experiment_accession', 'experiment_title',
       'experiment_desc', 'organism_taxid ', 'organism_name',
       'library_strategy', 'library_source', 'library_selection',
       'sample_accession', 'sample_title', 'instrument', 'total_spots',
       'total_size', 'run_accession', 'run_total_spots', 'run_total_bases',
       'run_alias', 'sra_url_alt1', 'sra_url_alt2', 'sra_url',
       'experiment_alias', 'ENA-FIRST-PUBLIC', 'ENA-LAST-UPDATE',
       'sra_url_alt', 'collection_date',
       'geographic location (country and/or sea)',
       'geographic location (region and locality)', 'ena_fastq_url',
       'ena_fastq_ftp', 'collecting institution', 'collection date',
       'collector name', 'definition for seropositive sample',
       'geographic location (latitude)', 'geographic location (longitude)',
       'host age', 'host behaviour', 'host common name', 'host health state',
       'host scientific name', 'host sex', 'host subject id', 'isolate',
     

In [32]:
# This is necessary as some files produced by pysradb contain three lines for some reason
columns = ['study_accession','experiment_accession','experiment_title','experiment_desc','organism_taxid','library_strategy','library_source','library_selection','sample_accession','instrument','total_size','run_accession','collection_date','location','host']

if len(srr_df) > 0:
    srr_df.dropna(subset=['study_accession'], inplace=True)
    srr_clean = srr_df[['study_accession','experiment_accession','experiment_title','experiment_desc','organism_taxid ','library_strategy','library_source','library_selection','sample_accession','instrument','total_size','run_accession','collection_date','geo_loc_name','host']]
    srr_clean.columns = columns
if len(err_df) > 0:
    err_df.dropna(subset=['study_accession'], inplace=True)
    err_clean = err_df[['study_accession','experiment_accession','experiment_title','experiment_desc','organism_taxid ','library_strategy','library_source','library_selection','sample_accession','instrument','total_size','run_accession','collection_date','geographic location (country and/or sea)','host scientific name']]
    err_clean.columns = columns

In [35]:
pd.concat([srr_clean,err_clean]).to_csv('current_md.txt',sep='\t')